# Network parsimony

The small parsimony problem is typically defined over phylogenetic trees. First assuming that traits are inherited independently of each other, we consider the problem for a single trait with possible characters in set $C$. The input of the small tree parsimony problem
is a tree and a labeling of the leaves by characters in $C$. The 
problem asks for a labeling of all tree nodes that minimizes the number of label differences ("mutations") over all tree edges.

Network parsimony generalizes the notion of parsimony from trees to networks.

We consider three types of small network parsimony problems the literature:

* hardwired parsimony
* softwired parsimony
* parental parsimony

In all of these problems one is given a rooted network, i.e. a finite directed acyclic graph (DAG) $G=(V,E)$ with one distinct root node $v_r$ together with a labeling of the leaves (or generally a partial labeling $\phi: V\rightarrow 2^C$. The task is to find a labeling of the nodes that minimizes a specific parsimony criterion.

In our considerations we follow the popular restriction to networks with indegree of at most two.

## Hardwired parsimony

Given a rooted network and partial node labeling, one minimizes the number of mutations over all directed edges of the network, i.e.

$PS_{hardwired} = \min_\psi \sum_{(u,v)\in E} \psi(u)\neq\psi(v)$

where $\psi: V\rightarrow C$ s.t. $\forall v\in V: \psi(v)\in\phi(v)$.

This is a formal extension of tree parsimony to networks; one accounts for mutations along all edges.

## Softwired parsimony

Given a rooted network and partial node labeling, one minimizes the number of mutations over all directed edges of a spanning tree of the network, i.e.

$PS_{softwired} = \min_{\psi, T} \sum_{(u,v)\in E_T} \psi(u)\neq\psi(v),$

where $\psi: V\rightarrow C$ s.t. $\forall v\in V: \psi(v)\in\phi(v)$.

The idea of this model is that the network specifies the set of potential phylogenetic trees over wich one solve the small tree parsimony problem. 

## Parental parsimony

This model considers the inheritance of allels in a polyploid setting. Characters can be mutated in children and/or collected from the ancestors (by allopoloploidy). (We present the problem from the lineage perspective!)
Consequently, nodes are labeled by sets of characters. 

$PS_{parental} = \min_{f} \sum_{v\in V} \operatorname{cost}_f(v),$

where
 * $f: V\rightarrow 2^C$
 * $f(v_r)| = 1$
 * $|f(v)| \leq \sum_{\text{$u$ parent of $v$}} |f(v)|$ 
 * $\operatorname{cost}_f(v) = |f(v) \setminus 
     \bigcup_{\text{$u$ parent of $v$}} f(u)|$

In [ ]:
import infrared as ir

In [ ]:
class DAG:
    """A simple class reprenting a rooted DAG
    """
    
    def __init__(self, numnodes, edges, characters):
        self._nodes = list(range(numnodes))
        self._edges = edges
        self._characters = characters
        
        # precompute parents, children, root
        # and perform some checks for 'proper' DAG
        self._parents=[[] for _ in self._nodes]
        for u,v in self._edges:
            self._parents[v].append(u)
    
        self._children=[[] for _ in self._nodes]
        for u,v in self._edges:
            self._children[u].append(v)
    
        assert(self._parents[0]==[])
        for node in self._nodes[1:]:
            assert(self._parents!=[])
    
        # find root
        self._root = None
        for v in self.nodes:
            if self.parents(v) is []:
                assert(self._root==None)
                self._root = v
    
    def parents(self, node):
        return self._parents[node]
    
    def children(self, node):
        return self._children[node]
        
    @property
    def root(self):
        return self._root
        
    @property
    def edges(self):
        return self._edges

    @property
    def nodes(self):
        return self._nodes

    @property
    def reticulation_nodes(self):
        return [v for v in self._nodes if len(self.parents(v))>1]
    
    @property
    def nonreticulation_nodes(self):
        return [v for v in self._nodes if len(self.parents(v))==1]

    @property
    def leaves(self):
        return [v for v in self._nodes if self.children(v)==[]]
    
    def show(self, labels):
        import graphviz
        
        G=graphviz.Digraph(engine="dot")
        
        for v in self.nodes:
            G.node(str(v), label=self._characters[labels[v]])
        for v,u in self.edges:
            G.edge(str(v),str(u))
        return G
    
    def print(self, labels):
        for v in self.nodes:
            print(f'{v}\'{self._characters[labels[v]]}\' -> {self.children(v)}')

## An example input instance

In [ ]:
characters = ["A","C","G","T"]

network = DAG(12,[(0,1),      (0,2),
                  (1,3),      (1,4),(2,4),(2,5),
                  (3,6),(3,7),(4,8),(4,9),(5,10),(5,11)],characters)

leaves = ['A','T','T','C','G','G']

## Model hardwired parsimony

### Define a hamming distance function between nodes 

In [ ]:
ir.def_function_class("HDistance",
    lambda u,v: [u,v],
    lambda psi_u,psi_v: psi_u != psi_v
    )

### Construct the model

In [ ]:
model = ir.Model()

model.add_variables(len(network.nodes), len(characters))
model.add_functions([HDistance(u,v) for u,v in network.edges],
                   'score')

model.set_feature_weight(-1, 'score') # -1, since we are minimizing

# constrain leaves
for i,leave in enumerate(network.leaves):
    val = characters.index(leaves[i])
    model.restrict_domains(leave, (val,val))

### Run optimization

In [ ]:
optimizer = ir.Optimizer(model)

best_score = -optimizer.evaluate()
best = optimizer.optimize()

print(f'Score: {best_score}')

network.show(best.values())

## Model softwired parsimony

### Define hamming distance for reticualtions

In [ ]:
ir.def_function_class("ReticulationHDistance",
    lambda u0,u1,v: [u0, u1, v],
    lambda psi_u0,psi_u1,psi_v: min(psi_u0 != psi_v,psi_u1 != psi_v)
    )

### Construct model

In [ ]:
model = ir.Model()

model.add_variables(len(network.nodes), len(characters))
model.add_functions([HDistance(network.parents(v)[0],v)
                     for v in network.nonreticulation_nodes],
                    'score')

model.add_functions([ReticulationHDistance(network.parents(v)[0],network.parents(v)[1],v)
                     for v in network.reticulation_nodes],
                    'score')

model.set_feature_weight(-1, 'score')

for i,leave in enumerate(network.leaves):
    val = characters.index(leaves[i])
    model.restrict_domains(leave, (val,val))

### Run optimization

In [ ]:
optimizer = ir.Optimizer(model)

best_score = -optimizer.evaluate()
best = optimizer.optimize()

print(f'Score: {best_score}')
network.print(best.values())